In [19]:
import pandas as pd

books_df = pd.read_csv('books.csv', encoding='ISO-8859-1', on_bad_lines='skip', delimiter=";",
                       dtype={'ISBN': 'str', "Book-Title": 'str', "Book-Author": 'str',
                              "Year-Of-Publication": 'str', "Publisher": 'str',
                              "Image-URL-S": 'str', "Image-URL-M": 'str', "Image-URL-L": 'str'})
books_df.columns = ['isbn', "title", "author", "year", "publisher", "image_s", "image_m", "image_l"]
books_df = books_df.drop_duplicates(['title'])

In [34]:
books_ratings_df = pd.read_csv('books_ratings.csv', encoding='ISO-8859-1', on_bad_lines='skip', delimiter=";")

In [35]:
print(list(books_ratings_df.columns.values))

['User-ID', 'ISBN', 'Book-Rating']


In [36]:
books_ratings_df.columns = ['userId', "isbn", "rating"]
books_ratings_df = books_ratings_df[books_ratings_df["rating"] > 0]
books_ratings_df = books_ratings_df[books_ratings_df.isbn.isin(set(books_df.isbn.values))]

unique_isbn_set = set(books_df.isbn.values)
isbnToIdx = {}
currentId = 0

for isbn in unique_isbn_set:
    isbnToIdx[isbn] = currentId
    currentId += 1

books_df['bookId'] = books_df.apply(lambda row: isbnToIdx[row.isbn], axis=1)
books_ratings_df["bookId"] = books_ratings_df.isbn.map(isbnToIdx)
# books_ratings_df = books_ratings_df.dropna(axis="rows", inplace=True)
# books_ratings_df = books_ratings_df.astype({'bookId': int})

In [43]:
books_ratings_df[books_ratings_df["userId"] < 100000]

,userId,isbn,rating,bookId
9563,8,0002005018,5,210954
9571,8,074322678X,5,204310
9575,8,0887841740,5,124639
9576,8,1552041778,5,115766
9578,8,1567407781,6,133464
...,...,...,...,...
418968,99996,1853260290,7,42452
418971,99997,0316666343,9,87763
418972,99997,0345443284,8,25009
418976,99997,0451202341,8,125234


In [ ]:
books_cb_df = pd.read_csv('books_cb.csv')
books_cb_df.head()

In [ ]:
titleToBookId = {}

bookIdToDesc = {}
bookIdToCategories = {}

In [ ]:


def update_row(row):
    titleToBookId[row.title] = row.bookId
    return row


books_df.apply(update_row, axis=1)

In [149]:
books_cb_df['bookId'] = books_cb_df.isbn10.map(isbnToIdx)

books_cb_df['bookId'] = books_cb_df.bookId.fillna(books_cb_df.title.map(titleToBookId))

books_cb_df = books_cb_df.dropna(subset=["bookId"])
books_cb_df = books_cb_df.astype({'bookId': int})

len(books_cb_df)

3617

In [ ]:
books_cb_union_df = books_df[
    books_df.bookId.isin(set(books_cb_df.bookId.values))].copy()
len(books_cb_union_df)

In [118]:
books_cb_df.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,bookId
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,149628
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,82494
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,114061
5,9780006280934,0006280935,The Problem of Pain,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,202915
6,9780006353287,0006353282,An Autobiography,NaN,Agatha Christie,"Authors, English",http://books.google.com/books/content?id=c49GQ...,Donation.,1977.0,4.27,560.0,3975.0,180484


In [155]:
for value in books_cb_df.values:
    bookIdToDesc[value[len(value) - 1]] = value[7]
    bookIdToCategories[value[len(value) - 1]] = value[5]

books_cb_union_df['categories'] = books_df.bookId.map(bookIdToCategories)
books_cb_union_df['description'] = books_df.bookId.map(bookIdToDesc)

3300


In [157]:

books_cb_union_df = books_cb_union_df.dropna(subset=["categories", "description"])
print(len(books_cb_union_df))

3300


In [ ]:
books_cb_union_rating_df = books_ratings_df[books_ratings_df.bookId.isin(set(books_cb_union_df.bookId))]

In [165]:
len(books_ratings_df)

328235

In [ ]:
# books_ratings_df.to_csv("../books-ratings-edited.csv", index=False)
# books_df.to_csv("../books-info-edited.csv", index=False)
# books_cb_union_rating_df.to_csv("../books-cb-rating-edited.csv", index=False)
# books_cb_union_df.to_csv("../books-cb-edited.csv", index=False)
